In [8]:
# Dependencies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import os


In [9]:
# Authenticate with Kaggle and download CSV file with interest rate data

import kaggle
kaggle.api.authenticate()

kaggle.api.dataset_download_files('federalreserve/interest-rates', unzip=True)
os.rename('./index.csv','./rates-data.csv')


In [26]:
# NASDAQ API setup + data download

import nasdaqdatalink
from api_keys import nasdaq_api_key

dataset_code = 'MULTPL/SP500_PE_RATIO_MONTH'

nasdaq_url = f'https://data.nasdaq.com/api/v3/datasets/{dataset_code}.json?api_key={nasdaq_api_key}'

### alternate option: nasdaq_df = pd.DataFrame(nasdaqdatalink.get(dataset_code))

response = requests.get(nasdaq_url).json()

pe_df = pd.DataFrame(response['dataset']['data'])

pe_df.rename(columns={0:'date',1:'pe'},inplace=True)

pe_df.head(10)


,date,pe
0,2023-07-01,25.78
1,2023-06-30,25.76
2,2023-06-01,25.15
3,2023-05-31,24.20
4,2023-05-01,24.00
5,2023-04-01,23.86
6,2023-03-31,21.97
7,2023-03-01,22.97
8,2023-02-28,21.22
9,2023-02-01,23.62


In [27]:
# Create dataframe of downloaded rate data

rates_df = pd.read_csv('./rates-data.csv')
rates_df.head(10)


,Year,Month,Day,Federal Funds Target Rate,Federal Funds Upper Target,Federal Funds Lower Target,Effective Federal Funds Rate,Real GDP (Percent Change),Unemployment Rate,Inflation Rate
0,1954,7,1,NaN,NaN,NaN,0.80,4.6,5.8,NaN
1,1954,8,1,NaN,NaN,NaN,1.22,NaN,6.0,NaN
2,1954,9,1,NaN,NaN,NaN,1.06,NaN,6.1,NaN
3,1954,10,1,NaN,NaN,NaN,0.85,8.0,5.7,NaN
4,1954,11,1,NaN,NaN,NaN,0.83,NaN,5.3,NaN
5,1954,12,1,NaN,NaN,NaN,1.28,NaN,5.0,NaN
6,1955,1,1,NaN,NaN,NaN,1.39,11.9,4.9,NaN
7,1955,2,1,NaN,NaN,NaN,1.29,NaN,4.7,NaN
8,1955,3,1,NaN,NaN,NaN,1.35,NaN,4.6,NaN
9,1955,4,1,NaN,NaN,NaN,1.43,6.7,4.7,NaN
